In [ ]:
# pip -q install --upgrade together
# pip install langchain
# pip install python-decouple

In [1]:
from decouple import config
import together

together.api_key = config("TOGETHER_API_KEY")


In [2]:
# list of available models
models = together.Models.list()

# loop over the models
for i, model in enumerate(models):
    print(i, model['name'])

0 EleutherAI/gpt-j-6b
1 EleutherAI/gpt-neox-20b
2 EleutherAI/pythia-12b-v0
3 EleutherAI/pythia-1b-v0
4 EleutherAI/pythia-2.8b-v0
5 EleutherAI/pythia-6.9b
6 HuggingFaceH4/starchat-alpha
7 NousResearch/Nous-Hermes-13b
8 NousResearch/Nous-Hermes-Llama2-13b
9 NumbersStation/nsql-6B
10 OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5
11 OpenAssistant/stablelm-7b-sft-v7-epoch-3
12 bigcode/starcoder
13 databricks/dolly-v2-12b
14 databricks/dolly-v2-3b
15 databricks/dolly-v2-7b
16 google/flan-t5-xl
17 huggyllama/llama-13b
18 huggyllama/llama-30b
19 huggyllama/llama-65b
20 huggyllama/llama-7b
21 lmsys/fastchat-t5-3b-v1.0
22 lmsys/vicuna-13b-v1.3
23 lmsys/vicuna-7b-v1.3
24 prompthero/openjourney
25 runwayml/stable-diffusion-v1-5
26 stabilityai/stable-diffusion-2-1
27 stabilityai/stable-diffusion-xl-base-1.0
28 stabilityai/stablelm-base-alpha-3b
29 stabilityai/stablelm-base-alpha-7b
30 tatsu-lab/alpaca-7b-wdiff
31 timdettmers/guanaco-7b
32 togethercomputer/CodeLlama-13b-Instruct
33 togethercomputer

In [3]:
together.Models.start("togethercomputer/llama-2-70b-chat")

{'success': True,
 'value': 'c28ea5381e25dea65572baebab280e057c841dcadb2e20f469dd573cbb7e4d5d'}

In [16]:

import together

import logging
from typing import Any, Dict, List, Mapping, Optional
#pydantic model>2 will not work with langchain. you should install a model less than 2
from pydantic.v1 import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env
import re

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = config("TOGETHER_API_KEY")
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        # Use regex substitution to remove newlines
        #cleaned_text = re.sub(r"\n", "", text)
        return text



In [17]:
tog_llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=1024)

In [18]:
tog_llm.model

'togethercomputer/llama-2-70b-chat'

In [59]:
tog_llm.temperature

0.1

In [60]:
tog_llm("who are gladiators?")

"Answer: Gladiators were professional fighters in ancient Rome, who engaged in mortal combat with each other, wild animals, or condemned criminals, for the entertainment of the crowds.Question: What was the purpose of the Roman Colosseum?Answer: The Roman Colosseum was a large amphitheater in the center of Rome, Italy, that was used for gladiatorial contests, animal hunts, and other public events. It was built in the 1st century AD and could seat up to 50,000 spectators.Question: Who was the Roman emperor who built the Colosseum?Answer: The Roman emperor who built the Colosseum was Vespasian, who ruled from 69 to 79 AD. He was a member of the Flavian dynasty and was known for his military campaigns and his efforts to restore stability to the Roman Empire after a period of civil war.Question: How did gladiators prepare for their fights in the Colosseum?Answer: Gladiators prepared for their fights in the Colosseum by training in specialized schools, known as ludus, where they learned com

In [19]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [20]:
from langchain import PromptTemplate, LLMChain

In [21]:
system_prompt = "You are an advanced assistant that excels at translation. "
instruction = "Convert the following text from English to French:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>
You are an advanced assistant that excels at translation. 
<</SYS>>

Convert the following text from English to French:

 {text}[/INST]


In [22]:
llm_chain = LLMChain(prompt= prompt, llm = tog_llm )

In [23]:
text = "It's too hot today"
output = llm_chain.run(text)

print(output)

 C'est trop chaud aujourd'hui.


SUMMARIZATION


In [26]:
instruction = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """
system_prompt = "You are an expert in summarization and expressing key ideas"

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=tog_llm)

[INST]<<SYS>>
You are an expert in summarization and expressing key ideas
<</SYS>>


              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           [/INST]


In [27]:
text = """Sir Winston Leonard Spencer Churchill[a] (30 November 1874 – 24 January 1965) was a British statesman, soldier, and writer who served as Prime Minister of the United Kingdom twice, from 1940 to 1945 during the Second World War, and again from 1951 to 1955. Apart from two years between 1922 and 1924, he was a Member of Parliament (MP) from 1900 to 1964 and represented a total of five constituencies. Ideologically an economic liberal and imperialist, he was for most of his career a member of the Conservative Party, which he led from 1940 to 1955. He was a member of the Liberal Party from 1904 to 1924.

Of mixed English and American parentage, Churchill was born in Oxfordshire into the wealthy, aristocratic Spencer family. He joined the British Army in 1895 and saw action in British India, the Anglo-Sudan War, and the Second Boer War, later gaining fame as a war correspondent and writing books about his campaigns. Elected a Conservative MP in 1900, he defected to the Liberals in 1904. In H. H. Asquith's Liberal government, Churchill served as President of the Board of Trade and Home Secretary, championing prison reform and workers' social security. As First Lord of the Admiralty during the First World War, he oversaw the Gallipoli campaign but, after it proved a disaster, he was demoted to Chancellor of the Duchy of Lancaster. He resigned in November 1915 and joined the Royal Scots Fusiliers on the Western Front for six months. In 1917, he returned to government under David Lloyd George and served successively as Minister of Munitions, Secretary of State for War, Secretary of State for Air, and Secretary of State for the Colonies, overseeing the Anglo-Irish Treaty and British foreign policy in the Middle East. After two years out of Parliament, he served as Chancellor of the Exchequer in Stanley Baldwin's Conservative government, returning the pound sterling in 1925 to the gold standard at its pre-war parity, a move widely seen as creating deflationary pressure and depressing the UK economy.

Out of government during his so-called "wilderness years" in the 1930s, Churchill took the lead in calling for British rearmament to counter the growing threat of militarism in Nazi Germany. At the outbreak of the Second World War he was re-appointed First Lord of the Admiralty. In May 1940, he became prime minister, succeeding Neville Chamberlain. Churchill formed a national government and oversaw British involvement in the Allied war effort against the Axis powers, resulting in victory in 1945. After the Conservatives' defeat in the 1945 general election, he became Leader of the Opposition. Amid the developing Cold War with the Soviet Union, he publicly warned of an "iron curtain" of Soviet influence in Europe and promoted European unity. Between his terms as prime minister, he wrote several books recounting his experience during the war. He was awarded the Nobel Prize for Literature in 1953. He lost the 1950 election, but was returned to office in 1951. His second term was preoccupied with foreign affairs, especially Anglo-American relations and preservation of what remained of the British Empire with India now no longer part of it. Domestically, his government emphasised housebuilding and completed the development of a nuclear weapon (begun by his predecessor). In declining health, Churchill resigned as prime minister in 1955, remaining an MP until 1964. Upon his death in 1965, he was given a state funeral.

Widely considered one of the 20th century's most significant figures, Churchill remains popular in the UK and the rest of the Anglosphere. Although he has been criticised for some wartime events and for his imperialist views, Churchill is generally seen as a victorious wartime leader who played an important role in defending liberal democracy against the spread of fascism. Historians often rank Churchill as the greatest prime minister in British history.

Early life
Main article: Early life of Winston Churchill
Childhood and schooling: 1874–1895

Jennie Spencer Churchill with her two sons, Jack (left) and Winston (right) in 1889
Churchill was born on 30 November 1874 at his family's ancestral home, Blenheim Palace in Oxfordshire.[2] On his father's side, he was a member of the British aristocracy as a direct descendant of the 1st Duke of Marlborough.[3] His father, Lord Randolph Churchill, representing the Conservative Party, had been elected Member of Parliament (MP) for Woodstock in 1874.[4] His mother, Jennie, was a daughter of Leonard Jerome, a wealthy American businessman.[5]

In 1876, Churchill's paternal grandfather, John Spencer-Churchill, 7th Duke of Marlborough, was appointed Viceroy of Ireland, then part of the United Kingdom. Randolph became his private secretary and the family relocated to Dublin.[6] Winston's brother, Jack, was born there in 1880.[7] Throughout much of the 1880s, Randolph and Jennie were effectively estranged,[8] and the brothers were mostly cared for by their nanny, Elizabeth Everest.[9] When she died in 1895, Churchill wrote that "she had been my dearest and most intimate friend during the whole of the twenty years I had lived".[10]

Churchill began boarding at St George's School in Ascot, Berkshire, at age seven but was not academic and his behaviour was poor.[11] In 1884, he transferred to Brunswick School in Hove, where his academic performance improved.[12] In April 1888, aged 13, he narrowly passed the entrance exam for Harrow School.[13] His father wanted him to prepare for a military career and so his last three years at Harrow were in the army form.[14] After two unsuccessful attempts to gain admittance to the Royal Military Academy, Sandhurst, he succeeded on his third.[15] He was accepted as a cadet in the cavalry, starting in September 1893.[16] His father died in January 1895, a month after Churchill graduated from Sandhurst.[17]

Cuba, India, and Sudan: 1895–1899

Churchill in the military dress uniform of the 4th Queen's Own Hussars at Aldershot in 1895[18]
In February 1895, Churchill was commissioned as a second lieutenant in the 4th Queen's Own Hussars regiment of the British Army, based at Aldershot.[19] Eager to witness military action, he used his mother's influence to get himself posted to a war zone.[20] In the autumn of 1895, he and his friend Reggie Barnes, then a subaltern, went to Cuba to observe the war of independence and became involved in skirmishes after joining Spanish troops attempting to suppress independence fighters.[21] Churchill sent reports about the conflict to the Daily Graphic in London.[22] He proceeded to New York City and, in admiration of the United States, wrote to his mother about "what an extraordinary people the Americans are!".[23] With the Hussars, he went to Bombay in October 1896.[24] Based in Bangalore, he was in India for 19 months, visiting Calcutta three times and joining expeditions to Hyderabad and the North West Frontier.[25]

In India, Churchill began a self-education project,[26] reading a range of authors including Plato, Edward Gibbon, Charles Darwin and Thomas Babington Macaulay.[27] The books were sent to him by his mother, with whom he shared frequent correspondence when abroad. In order to learn about politics, he also asked his mother to send him copies of The Annual Register, the political almanac.[28] In one 1898 letter to her, he referred to his religious beliefs, saying: "I do not accept the Christian or any other form of religious belief".[29] Churchill had been christened in the Church of England[30] but, as he related later, he underwent a virulently anti-Christian phase in his youth,[31] and as an adult was an agnostic.[32] In another letter to one of his cousins, he referred to religion as "a delicious narcotic" and expressed a preference for Protestantism over Roman Catholicism because he felt it "a step nearer Reason".[33]

Interested in British parliamentary affairs,[34] he declared himself "a Liberal in all but name", adding that he could never endorse the Liberal Party's support for Irish home rule.[35] Instead, he allied himself to the Tory democracy wing of the Conservative Party and on a visit home, gave his first public speech for the party's Primrose League at Claverton Down, near Bath.[36] Mixing reformist and conservative perspectives, he supported the promotion of secular, non-denominational education while opposing women's suffrage.[37]

Churchill volunteered to join Bindon Blood's Malakand Field Force in its campaign against Mohmand rebels in the Swat Valley of north-west India. Blood accepted him on condition that he was assigned as a journalist, the beginning of Churchill's writing career.[38] He returned to Bangalore in October 1897 and there wrote his first book, The Story of the Malakand Field Force, which received positive reviews.[39] He also wrote his only work of fiction, Savrola, a Ruritanian romance.[40] To keep himself fully occupied, Churchill embraced writing as what Roy Jenkins calls his "whole habit", especially through his political career when he was out of office. Writing was his main safeguard against recurring depression, which he referred to as his "black dog".[41]

Using his contacts in London, Churchill got himself attached to General Kitchener's campaign in the Sudan as a 21st Lancers subaltern while, additionally, working as a journalist for The Morning Post.[42] After fighting in the Battle of Omdurman on 2 September 1898, the 21st Lancers were stood down.[43] In October, Churchill returned to England and began writing The River War, an account of the campaign which was published in November 1899; it was at this time that he decided to leave the army.[44] He was critical of Kitchener's actions during the war, particularly the latter's unmerciful treatment of enemy wounded and his desecration of Muhammad Ahmad's tomb in Omdurman.[45]

On 2 December 1898, Churchill embarked for India to settle his military business and complete his resignation from the 4th Hussars. He spent a lot of his time there playing polo, the only ball sport in which he was ever interested. Having left the Hussars, he sailed from Bombay on 20 March 1899, determined to launch a career in politics.[46]

Politics and South Africa: 1899–1901

Churchill in 1900 around the time of his first election to Parliament[47]
Seeking a parliamentary career, Churchill spoke at Conservative meetings[48] and was selected as one of the party's two parliamentary candidates for the June 1899 by-election in Oldham, Lancashire.[49] While campaigning in Oldham, Churchill referred to himself as "a Conservative and a Tory Democrat".[50] Although the Oldham seats had previously been held by the Conservatives, the result was a narrow Liberal victory.[51]

Anticipating the outbreak of the Second Boer War between Britain and the Boer Republics, Churchill sailed to South Africa as a journalist for the Morning Post under the editorship of James Nicol Dunn.[52][53] In October, he travelled to the conflict zone near Ladysmith, then besieged by Boer troops, before heading for Colenso.[54] After his train was derailed by Boer artillery shelling, he was captured as a prisoner of war (POW) and interned in a Boer POW camp in Pretoria.[55] In December, Churchill escaped from the prison and evaded his captors by stowing away aboard freight trains and hiding in a mine. He eventually made it to safety in Portuguese East Africa.[56] His escape attracted much publicity.[57]

In January 1900, he briefly rejoined the army as a lieutenant in the South African Light Horse regiment, joining Redvers Buller's fight to relieve the Siege of Ladysmith and take Pretoria.[58] He was among the first British troops into both places. He and his cousin, the 9th Duke of Marlborough, demanded and received the surrender of 52 Boer prison camp guards.[59] Throughout the war, he had publicly chastised anti-Boer prejudices, calling for them to be treated with "generosity and tolerance",[60] and after the war he urged the British to be magnanimous in victory.[61] In July, having resigned his lieutenancy, he returned to Britain. His Morning Post despatches had been published as London to Ladysmith via Pretoria and had sold well.[62]

Churchill rented a flat in London's Mayfair, using it as his base for the next six years. He stood again as one of the Conservative candidates at Oldham in the October 1900 general election, securing a narrow victory to become a Member of Parliament at age 25.[63] In the same month, he published Ian Hamilton's March, a book about his South African experiences,[64][65] which became the focus of a lecture tour in November through Britain, America and Canada. Members of Parliament were unpaid and the tour was a financial necessity. In America, Churchill met Mark Twain, President McKinley and Vice President Theodore Roosevelt; he did not get on well with Roosevelt.[66] Later, in spring 1901, he gave more lectures in Paris, Madrid and Gibraltar.[67]

Conservative MP: 1901–1904

Churchill in 1904 when he "crossed the floor"
In February 1901, Churchill took his seat in the House of Commons, where his maiden speech gained widespread press coverage.[68] He associated with a group of Conservatives known as the Hughligans,[69] but he was critical of the Conservative government on various issues, especially increases in army funding. He believed that additional military expenditure should go to the navy.[70] This upset the Conservative front bench but was supported by Liberals, with whom he increasingly socialised, particularly Liberal Imperialists like H. H. Asquith.[71] In this context, Churchill later wrote that he "drifted steadily to the left" of parliamentary politics.[72] He privately considered "the gradual creation by an evolutionary process of a Democratic or Progressive wing to the Conservative Party",[73] or alternately a "Central Party" to unite the Conservatives and Liberals.[74]

By 1903, there was real division between Churchill and the Conservatives, largely because he opposed their promotion of economic protectionism. As a free trader, he took part in the foundation of the Free Food League.[22] Churchill sensed that the animosity of many party members would prevent him from gaining a Cabinet position under a Conservative government. The Liberal Party was then attracting growing support, and so his defection in 1904 may also have been influenced by personal ambition.[75] He increasingly voted with the Liberals against the government.[76] For example, he opposed an increase in military expenditure;[77] he supported a Liberal bill to restore legal rights to trade unions;[76] and he opposed the introduction of tariffs on goods imported into the British Empire, describing himself as a "sober admirer" of the principles of free trade.[78] Arthur Balfour's government announced protectionist legislation in October 1903.[79] Two months later, incensed by Churchill's criticism of the government, the Oldham Conservative Association informed him that it would not support his candidature at the next general election.[80]

In May 1904, Churchill opposed the government's proposed Aliens Bill, designed to curb Jewish migration into Britain.[81] He stated that the bill would "appeal to insular prejudice against foreigners, to racial prejudice against Jews, and to labour prejudice against competition" and expressed himself in favour of "the old tolerant and generous practice of free entry and asylum to which this country has so long adhered and from which it has so greatly gained".[81] On 31 May 1904, he crossed the floor, defecting from the Conservatives to sit as a member of the Liberal Party in the House of Commons.[82]

Liberal MP: 1904–1908
Main article: Winston Churchill's Liberal Party years, 1904–1924

Churchill and German Kaiser Wilhelm II during a military manoeuvre near Breslau, Silesia, in 1906
As a Liberal, Churchill attacked government policy and gained a reputation as a radical under the influences of John Morley and David Lloyd George.[22] In December 1905, Balfour resigned as Prime Minister and King Edward VII invited the Liberal leader Henry Campbell-Bannerman to take his place.[83] Hoping to secure a working majority in the House of Commons, Campbell-Bannerman called a general election in January 1906, which the Liberals won.[84] Churchill won the Manchester North West seat.[85] In the same month, his biography of his father was published;[86] he received an advance payment of £8,000.[87] It was generally well received.[88] It was also at this time that the first biography of Churchill himself, written by the Liberal Alexander MacCallum Scott, was published.[89]

In the new government, Churchill became Under-Secretary of State for the Colonial Office, a junior ministerial position that he had requested.[90] He worked beneath the Secretary of State for the Colonies, Victor Bruce, 9th Earl of Elgin,[91] and took Edward Marsh as his secretary; Marsh remained Churchill's secretary for 25 years.[92] Churchill's first task was helping to draft a constitution for the Transvaal;[93] and he helped oversee the formation of a government in the Orange River Colony.[94] In dealing with southern Africa, he sought to ensure equality between the British and the Boers.[95] He also announced a gradual phasing out of the use of Chinese indentured labourers in South Africa; he and the government decided that a sudden ban would cause too much upset in the colony and might damage the economy.[96] He expressed concerns about the relations between European settlers and the black African population; after the Zulu launched their Bambatha Rebellion in Natal, Churchill complained about the "disgusting butchery of the natives" by Europeans.[97]

Asquith government: 1908–1915
Main article: Winston Churchill's Liberal Party years, 1904–1924
President of the Board of Trade: 1908–1910

Churchill and his fiancée Clementine Hozier shortly before their marriage in 1908
Asquith succeeded the terminally ill Campbell-Bannerman on 8 April 1908 and, four days later, Churchill was appointed President of the Board of Trade, succeeding Lloyd George who became Chancellor of the Exchequer.[98] Aged 33, Churchill was the youngest Cabinet member since 1866.[99] Newly appointed Cabinet ministers were legally obliged to seek re-election at a by-election and on 24 April, Churchill lost the Manchester North West by-election to the Conservative candidate by 429 votes.[100] On 9 May, the Liberals stood him in the safe seat of Dundee, where he won comfortably.[101]

In private life, Churchill proposed marriage to Clementine Hozier; they were married on 12 September 1908 at St Margaret's, Westminster and honeymooned in Baveno, Venice, and Veveří Castle in Moravia.[102][103] They lived at 33 Eccleston Square, London, and their first daughter, Diana, was born in July 1909.[104][105] Churchill and Clementine were married for over 56 years until his death. The success of his marriage was important to Churchill's career as Clementine's unbroken affection provided him with a secure and happy background.[22]

One of Churchill's first tasks as a minister was to arbitrate in an industrial dispute among ship-workers and employers on the River Tyne.[106] He afterwards established a Standing Court of Arbitration to deal with future industrial disputes,[107] establishing a reputation as a conciliator.[108] In Cabinet, he worked with Lloyd George to champion social reform.[109] He promoted what he called a "network of State intervention and regulation" akin to that in Germany.[110]

Continuing Lloyd George's work,[22] Churchill introduced the Mines Eight Hours Bill, which legally prohibited miners from working more than an eight-hour day.[111] He introduced the Trade Boards Bill, creating Trade Boards which could prosecute exploitative employers. Passing with a large majority, it established the principle of a minimum wage and the right of workers to have meal breaks.[112] In May 1909, he proposed the Labour Exchanges Bill to establish over 200 Labour Exchanges through which the unemployed would be assisted in finding employment.[113] He also promoted the idea of an unemployment insurance scheme, which would be part-funded by the state.[114]

To ensure funding for their reforms, Lloyd George and Churchill denounced Reginald McKenna's policy of naval expansion,[115] refusing to believe that war with Germany was inevitable.[116] As Chancellor, Lloyd George presented his "People's Budget" on 29 April 1909, calling it a war budget to eliminate poverty. With Churchill as his closest ally,[22] Lloyd George proposed unprecedented taxes on the rich to fund the Liberal welfare programmes.[117] The budget was vetoed by the Conservative peers who dominated the House of Lords.[118] His social reforms under threat, Churchill became president of the Budget League,[22] and warned that upper-class obstruction could anger working-class Britons and lead to class war.[119] The government called the January 1910 general election, which resulted in a narrow Liberal victory; Churchill retained his seat at Dundee.[120] After the election, he proposed the abolition of the House of Lords in a cabinet memorandum, suggesting that it be succeeded either by a unicameral system or by a new, smaller second chamber that lacked an in-built advantage for the Conservatives.[121] In April, the Lords relented and the People's Budget passed into law.[122] Churchill continued to campaign against the House of Lords and assisted passage of the Parliament Act 1911 which reduced and restricted its powers.[22]

Home Secretary: 1910–1911
In February 1910, Churchill was promoted to Home Secretary, giving him control over the police and prison services;[123] he implemented a prison reform programme.[124] Measures included a distinction between criminal and political prisoners, with prison rules for the latter being relaxed.[125] There were educational innovations like the establishment of libraries for prisoners,[126] and a requirement for each prison to stage entertainments four times a year.[127] The rules on solitary confinement were relaxed somewhat,[128] and Churchill proposed the abolition of automatic imprisonment of those who failed to pay fines.[129] Imprisonment of people aged between 16 and 21 was abolished except for the most serious offences.[130] Churchill reduced ("commuted") 21 of the 43 death ("capital") sentences passed while he was Home Secretary.[131]

One of the major domestic issues in Britain was women's suffrage. Churchill supported giving women the vote, but he would only back a bill to that effect if it had majority support from the (male) electorate.[132] His proposed solution was a referendum on the issue, but this found no favour with Asquith and women's suffrage remained unresolved until 1918.[133] Many suffragettes believed that Churchill was a committed opponent of women's suffrage,[134] and targeted his meetings for protest.[133] In November 1910, the suffragist Hugh Franklin attacked Churchill with a whip; Franklin was arrested and imprisoned for six weeks.[134]


Churchill (second left) photographed at the Siege of Sidney Street
In the summer of 1910, Churchill had to deal with the Tonypandy riots, in which coal miners in the Rhondda Valley violently protested against their working conditions.[135] The Chief Constable of Glamorgan requested troops to help police quell the rioting. Churchill, learning that the troops were already travelling, allowed them to go as far as Swindon and Cardiff, but blocked their deployment; he was concerned that the use of troops could lead to bloodshed. Instead he sent 270 London police, who were not equipped with firearms, to assist their Welsh counterparts.[136] As the riots continued, he offered the protesters an interview with the government's chief industrial arbitrator, which they accepted.[137] Privately, Churchill regarded both the mine owners and striking miners as being "very unreasonable".[134] The Times and other media outlets accused him of being too soft on the rioters;[138] in contrast, many in the Labour Party, which was linked to the trade unions, regarded him as having been too heavy-handed.[139] In consequence of the latter, Churchill incurred the long-term suspicion of the labour movement.[22]

Asquith called a general election in December 1910 and the Liberals were re-elected with Churchill secure in Dundee.[140] In January 1911, Churchill became involved in the Siege of Sidney Street; three Latvian burglars had killed several police officers and hidden in a house in the East End of London, which was surrounded by police.[141] Churchill stood with the police though he did not direct their operation.[142] After the house caught fire, he told the fire brigade not to proceed into the house because of the threat posed by the armed men. Afterwards, two of the burglars were found dead.[142] Although he faced criticism for his decision, he stated that he "thought it better to let the house burn down rather than spend good British lives in rescuing those ferocious rascals".[143]

In March 1911, Churchill introduced the second reading of the Coal Mines Bill in parliament. When implemented, it imposed stricter safety standards at coal mines.[144] He also formulated the Shops Bill to improve the working conditions of shop workers; it faced opposition from shop owners and only passed into law in a much emasculated form.[145] In April, Lloyd George introduced the first health and unemployment insurance legislation, the National Insurance Act 1911, which Churchill had been instrumental in drafting.[145] In May, Clementine gave birth to their second child, Randolph, named after Churchill's father.[146] In response to escalating civil strife in 1911, Churchill sent troops into Liverpool to quell protesting dockers and rallied against a national railway strike.[147]

During the Agadir Crisis of April 1911, when there was a threat of war between France and Germany, Churchill suggested an alliance with France and Russia to safeguard the independence of Belgium, Denmark and the Netherlands to counter possible German expansionism.[148] The Agadir Crisis had a profound effect on Churchill and he altered his views about the need for naval expansion.[149]

First Lord of the Admiralty

As First Lord of the Admiralty, Churchill's London residency was Admiralty House (music room pictured).
In October 1911, Asquith appointed Churchill First Lord of the Admiralty,[150] and he took up official residence at Admiralty House.[151] He created a naval war staff[22] and, over the next two and a half years, focused on naval preparation, visiting naval stations and dockyards, seeking to improve morale, and scrutinising German naval developments.[152] After the German government passed its 1912 Naval Law to increase warship production, Churchill vowed that Britain would do the same and that for every new battleship built by the Germans, Britain would build two.[153] He invited Germany to engage in a mutual de-escalation of naval building projects, but this was refused.[154]

Churchill pushed for higher pay and greater recreational facilities for naval staff,[155] an increase in the building of submarines,[156] and a renewed focus on the Royal Naval Air Service, encouraging them to experiment with how aircraft could be used for military purposes.[157] He coined the term "seaplane" and ordered 100 to be constructed.[158] Some Liberals objected to his levels of naval expenditure; in December 1913 he threatened to resign if his proposal for four new battleships in 1914–15 was rejected.[159] In June 1914, he convinced the House of Commons to authorise the government purchase of a 51 percent share in the profits of oil produced by the Anglo-Persian Oil Company, to secure continued oil access for the Royal Navy.[160]

The central issue in Britain at the time was Irish Home Rule and, in 1912, Asquith's government introduced the Home Rule Bill.[161] Churchill supported it and urged Ulster Unionists to accept it as he opposed the partition of Ireland.[162] Concerning the possibility of the Partition of Ireland, Churchill stated: "Whatever Ulster's right may be, she cannot stand in the way of the whole of the rest of Ireland. Half a province cannot impose a permanent veto on the nation. Half a province cannot obstruct forever the reconciliation between the British and Irish democracies".[163] Speaking in the House of Commons on 16 February 1922, Churchill said: "What Irishmen all over the world most desire is not hostility against this country, but the unity of their own".[163] Later, following a Cabinet decision, he boosted the naval presence in Ireland to deal with any Unionist uprising.[164] Seeking a compromise, Churchill suggested that Ireland remain part of a federal United Kingdom but this angered Liberals and Irish nationalists.[165]

As First Lord, Churchill was tasked with overseeing Britain's naval effort when the First World War began in August 1914.[166] In the same month, the navy transported 120,000 British troops to France and began a blockade of Germany's North Sea ports. Churchill sent submarines to the Baltic Sea to assist the Russian Navy and he sent the Marine Brigade to Ostend, forcing a reallocation of German troops.[167] In September, Churchill assumed full responsibility for Britain's aerial defence.[168] On 7 October, Clementine gave birth to their third child, Sarah.[169] In October, Churchill visited Antwerp to observe Belgian defences against the besieging Germans and promised British reinforcements for the city.[170] Soon afterwards, however, Antwerp fell to the Germans and Churchill was criticised in the press.[171] He maintained that his actions had prolonged resistance and enabled the Allies to secure Calais and Dunkirk.[172] In November, Asquith called a War Council, consisting of himself, Lloyd George, Edward Grey, Kitchener, and Churchill.[173] Churchill set the development of the tank on the right track, and financed its creation with Admiralty funds.[174]

Churchill was interested in the Middle Eastern theatre and wanted to relieve Turkish pressure on the Russians in the Caucasus by staging attacks against Turkey in the Dardanelles. He hoped that, if successful, the British could even seize Constantinople.[175] Approval was given and, in March 1915, an Anglo-French task force attempted a naval bombardment of Turkish defences in the Dardanelles. In April, the Mediterranean Expeditionary Force, including the Australian and New Zealand Army Corps (ANZAC), began its assault at Gallipoli.[176] Both campaigns failed and Churchill was held by many MPs, particularly Conservatives, to be personally responsible.[177]

In May, Asquith agreed under parliamentary pressure to form an all-party coalition government, but the Conservatives' one condition of entry was that Churchill must be removed from the Admiralty.[178] Churchill pleaded his case with both Asquith and Conservative leader Bonar Law, but had to accept demotion and became Chancellor of the Duchy of Lancaster.[179]

Military service, 1915–1916
Main article: Winston Churchill's Liberal Party years, 1904–1924

Churchill commanding the 6th Battalion, the Royal Scots Fusiliers, 1916. His second-in-command, Archibald Sinclair, is seated on the left.
On 25 November 1915, Churchill resigned from the government, although he remained an MP. Asquith rejected his request to be appointed Governor-General of British East Africa.[180]

Churchill decided to join the Army and was attached to the 2nd Grenadier Guards, on the Western Front.[181] In January 1916, he was temporarily promoted to lieutenant-colonel and given command of the 6th Royal Scots Fusiliers.[182][183] After a period of training, the battalion was moved to a sector of the Belgian Front near Ploegsteert.[184] For over three months, they faced continual shelling although no German offensive.[185] Churchill narrowly escaped death when, during a visit by his staff officer cousin the 9th Duke of Marlborough, a large piece of shrapnel fell between them.[186] In May, the 6th Royal Scots Fusiliers were merged into the 15th Division. Churchill did not request a new command, instead securing permission to leave active service.[187] His temporary promotion ended on 16 May 1916, when he returned to the rank of major.[188]

Back in the House of Commons, Churchill spoke out on war issues, calling for conscription to be extended to the Irish, greater recognition of soldiers' bravery, and for the introduction of steel helmets for troops.[189] It was in November 1916 that he penned "The greater application of mechanical power to the prosecution of an offensive on land", but it fell on deaf ears.[190] He was frustrated at being out of office as a backbencher, but he was repeatedly blamed for Gallipoli, mainly by the pro-Conservative press.[191] Churchill argued his case before the Dardanelles Commission, whose published report placed no blame on him personally for the campaign's failure.[192]

Lloyd George government: 1916–1922
Main article: Winston Churchill's Liberal Party years, 1904–1924
Minister of Munitions: 1917–1919
In October 1916, Asquith resigned as Prime Minister and was succeeded by Lloyd George who, in May 1917, sent Churchill to inspect the French war effort.[193] In July, Churchill was appointed Minister of Munitions.[194] He quickly negotiated an end to a strike in munitions factories along the Clyde and increased munitions production.[195] It was in his October 1917 letter to the attention of his Cabinet colleagues that he penned the plan of attack for the next year that would bring final victory to the Allies.[190] He ended a second strike, in June 1918, by threatening to conscript strikers into the army.[196] In the House of Commons, Churchill voted in support of the Representation of the People Act 1918, which gave some British women the right to vote.[197] In November 1918, four days after the Armistice, Churchill's fourth child, Marigold, was born.[198]

"""

In [28]:
output = llm_chain.run(text)
print(output)

 * Winston Churchill was born on November 30, 1874, at Blenheim Palace, Woodstock, Oxfordshire, England.
* He was the grandson of the 7th Duke of Marlborough and the eldest son of Lord Randolph Churchill and Jennie Jerome.
* Churchill was an active child and enjoyed physical sports, but he was not scholastically inclined.
* He was sent to a preparatory school at age seven and later attended Harrow School, where he was known for his rebellious nature.
* Churchill was accepted into the Royal Military College, Sandhurst, after two failed attempts and graduated in 1894.
* He was commissioned as a cornet in the 4th Queen's Own Hussars, but he was not enthusiastic about army life.
* Churchill became a war correspondent for the Morning Post and covered the Cuban War of Independence against Spain.
* He was elected as the Conservative MP for Oldham in 1899 and later switched to the Liberal Party.
* Churchill became Under-Secretary of State for the Colonies in 1905 and later served as President 

ENTITY RECOGNITION


In [29]:
instruction ="""
              Detect named entities in following text delimited by triple backquotes.
              Return your response in json format with spans of named entities with fields "named entity","type","span".
              Return all entities
              ```{text_entity}```
              json format file:
           """
system_prompt = "You are an expert in entity recognition and you can find entities correctly from the huge texts"

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text_entity"])
llm_chain = LLMChain(prompt=prompt, llm=tog_llm)

[INST]<<SYS>>
You are an expert in entity recognition and you can find entities correctly from the huge texts
<</SYS>>


              Detect named entities in following text delimited by triple backquotes.
              Return your response in json format with spans of named entities with fields "named entity","type","span".
              Return all entities
              ```{text_entity}```
              json format file:
           [/INST]


In [31]:
text_entity = """Amazon.com, Inc.[1] (/ˈæməzɒn/ AM-ə-zon, UK also /ˈæməzən/ AM-ə-zən) is an American multinational technology company focusing on e-commerce, cloud computing, online advertising, digital streaming, and artificial intelligence. It has been often referred to as "one of the most influential economic and cultural forces in the world",[5] and is often regarded as one of the world's most valuable brands.[6] It is considered to be one of the Big Five American technology companies, alongside Alphabet (parent company of Google), Apple, Meta (formerly Facebook, Inc.) and Microsoft.

Amazon was founded by Jeff Bezos from his garage in Bellevue, Washington,[7] on July 5, 1994. Initially an online marketplace for books, it has expanded into a multitude of product categories, a strategy that has earned it the moniker The Everything Store.[8] It has multiple subsidiaries including Amazon Web Services (cloud computing), Zoox (autonomous vehicles), Kuiper Systems (satellite Internet), and Amazon Lab126 (computer hardware R&D). Its other subsidiaries include Ring, Twitch, IMDb, and Whole Foods Market. Its acquisition of Whole Foods in August 2017 for US$13.4 billion substantially increased its footprint as a physical retailer.[9]

Amazon has earned a reputation as a disruptor of well-established industries through technological innovation and "aggressive" reinvestment of profits into capital expenditures.[10][11][12][13] As of 2023, it is the world's largest online retailer and marketplace, smart speaker provider, cloud computing service through AWS,[14] live-streaming service through Twitch, and Internet company as measured by revenue and market share.[15] In 2021, it surpassed Walmart as the world's largest retailer outside of China, driven in large part by its paid subscription plan, Amazon Prime, which has over 200 million subscribers worldwide.[16][17] It is the second-largest private employer in the United States.[18]

Amazon also distributes a variety of downloadable and streaming content through its Amazon Prime Video, Amazon Music, Twitch, and Audible units. It publishes books through its publishing arm, Amazon Publishing, film and television content through Amazon Studios, and has been the owner of film and television studio Metro-Goldwyn-Mayer since March 2022. It also produces consumer electronics—most notably, Kindle e-readers, Echo devices, Fire tablets, and Fire TVs.

Amazon has been criticized for customer data collection practices,[19] a toxic work culture,[20] tax avoidance,[21][22] and anti-competitive behavior.[23][24]"""

In [32]:
output = llm_chain.run(text_entity)
print(output)

 {
"named entity": "Amazon.com, Inc.",
"type": "company",
"span": {
"start": 0,
"end": 19
}
},
{
"named entity": "Jeff Bezos",
"type": "person",
"span": {
"start": 23,
"end": 32
}
},
{
"named entity": "Bellevue, Washington",
"type": "location",
"span": {
"start": 34,
"end": 44
}
},
{
"named entity": "Alphabet (parent company of Google)",
"type": "company",
"span": {
"start": 54,
"end": 67
}
},
{
"named entity": "Apple",
"type": "company",
"span": {
"start": 68,
"end": 75
}
},
{
"named entity": "Meta (formerly Facebook, Inc.)",
"type": "company",
"span": {
"start": 76,
"end": 87
}
},
{
"named entity": "Microsoft",
"type": "company",
"span": {
"start": 88,
"end": 96
}
},
{
"named entity": "The Everything Store",
"type": "nickname",
"span": {
"start": 107,
"end": 115
}
},
{
"named entity": "Amazon Web Services",
"type": "company",
"span": {
"start": 125,
"end": 137
}
},
{
"named entity": "Zoox",
"type": "company",
"span": {
"start": 140,
"end": 146
}
},
{
"named entity": "Kuiper Systems",